In [1]:
import pandas as pd

from src.prisma import prisma

In [2]:
async def startup():
    await prisma.connect()

async def shutdown():
    await prisma.disconnect()

In [3]:
await startup()

In [4]:
async def get_players_rounds_stats_by_match_id(match_id: str):
    match_rounds_raw_data = await prisma.round.find_many(
        where={
            'matchId': match_id,
        },
        include={
            'playerRoundStats': True
        },
    )

    match_rounds_players_stats = []
    for match_round_raw_data in match_rounds_raw_data:
        match_round_raw_data = match_round_raw_data.dict()

        match_round_players_stats = []
        for match_round_player_stats in match_round_raw_data["playerRoundStats"]:
            match_round_player_stats["matchId"] = match_round_raw_data["matchId"]
    
            match_round_players_stats.append(match_round_player_stats)

        match_rounds_players_stats += match_round_players_stats

    return match_rounds_players_stats

In [5]:
async def get_players_rounds_stats_by_match_id_df(match_id: str):
    match_rounds_players_stats = await get_players_rounds_stats_by_match_id(match_id)

    match_rounds_players_stats_df = pd.DataFrame(match_rounds_players_stats)

    return match_rounds_players_stats_df

In [6]:
match_rounds_players_stats_df = await get_players_rounds_stats_by_match_id_df("c5c7500c-6db8-45c8-b6ab-2be01b401ef7")

In [7]:
match_rounds_players_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      270 non-null    object
 1   playerViewTime          270 non-null    int64 
 2   LastPersonStandingTime  270 non-null    int64 
 3   viewTimePercentage      0 non-null      object
 4   Ace                     270 non-null    bool  
 5   Clutch                  270 non-null    bool  
 6   kills                   270 non-null    int64 
 7   killAssistsReceived     270 non-null    int64 
 8   killAssistsGiven        270 non-null    int64 
 9   selfKills               270 non-null    int64 
 10  deaths                  270 non-null    int64 
 11  headshots               270 non-null    int64 
 12  damageDealt             270 non-null    int64 
 13  damageTaken             270 non-null    int64 
 14  teamDamageDealt         270 non-null    int64 
 15  teamDa

In [8]:
async def get_players():
    players_raw_data = await prisma.player.find_many(
        distinct=["playerId"],
    )

    players = []
    for player_raw_data in players_raw_data:
        players.append(player_raw_data.dict())

    return players

In [9]:
async def get_team_ids_player_ids_df():
    players = await get_players()

    players_df = pd.DataFrame(players)

    team_ids_player_ids_df = players_df[["playerId", "teamId"]]

    return team_ids_player_ids_df

In [10]:
team_ids_player_ids_df = await get_team_ids_player_ids_df()

In [11]:
team_ids_player_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   playerId  10 non-null     object
 1   teamId    10 non-null     object
dtypes: object(2)
memory usage: 288.0+ bytes


In [12]:
async def get_teams_rounds_stats_by_match_id(match_id: str):
    match_rounds_raw_data = await prisma.round.find_many(
        where={
            'matchId': match_id,
        },
        include={
            'teamRoundStats': True
        },
    )

    match_rounds_teams_stats = []
    for match_round_raw_data in match_rounds_raw_data:
        match_round_raw_data = match_round_raw_data.dict()

        match_round_teams_stats = []
        for match_round_team_stats in match_round_raw_data["teamRoundStats"]:
            match_round_team_stats["matchId"] = match_round_raw_data["matchId"]
    
            match_round_teams_stats.append(match_round_team_stats)

        match_rounds_teams_stats += match_round_teams_stats

    return match_rounds_teams_stats

In [13]:
async def get_teams_rounds_stats_by_match_id_df(match_id: str):
    match_rounds_teams_stats = await get_teams_rounds_stats_by_match_id(match_id)

    match_rounds_teams_stats_df = pd.DataFrame(match_rounds_teams_stats)

    return match_rounds_teams_stats_df

In [14]:
match_rounds_teams_stats_df = await get_teams_rounds_stats_by_match_id_df("c5c7500c-6db8-45c8-b6ab-2be01b401ef7")

In [15]:
match_rounds_teams_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  54 non-null     object
 1   teamViewTime        54 non-null     int64 
 2   viewTimePercentage  0 non-null      object
 3   side                54 non-null     object
 4   won                 54 non-null     bool  
 5   winType             0 non-null      object
 6   kills               54 non-null     int64 
 7   deaths              54 non-null     int64 
 8   damageDealt         54 non-null     int64 
 9   damageTaken         54 non-null     int64 
 10  teamDamageOccured   54 non-null     int64 
 11  teamKillsOccured    54 non-null     int64 
 12  selfKills           54 non-null     int64 
 13  weaponKills         54 non-null     object
 14  headshots           54 non-null     int64 
 15  round               0 non-null      object
 16  roundId             54 non-n

In [19]:
async def get_aces_by_match_id(match_id: str):
    match_rounds_players_stats_df = await get_players_rounds_stats_by_match_id_df(match_id)
    team_ids_player_ids_df = await get_team_ids_player_ids_df()

    match_rounds_players_stats_df = match_rounds_players_stats_df.merge(team_ids_player_ids_df, on="playerId", how="inner")

    team_ids = match_rounds_players_stats_df["teamId"].unique()

    match_rounds_players_stats_df["opponentTeamId"] = match_rounds_players_stats_df.apply(
        lambda x: team_ids[1] if x["teamId"] == team_ids[0] else team_ids[0],
        axis=1
    )

    num_players = match_rounds_players_stats_df.groupby(
        ["roundId", "teamId"]
    )["playerId"].count().reset_index().rename(columns={"playerId": "playersCount"})

    match_rounds_players_stats_df = match_rounds_players_stats_df.merge(
        num_players, 
        left_on=["roundId", "opponentTeamId"],
        right_on=["roundId", "teamId"],
        how="inner"
    )

    match_rounds_players_stats_df.rename(columns={"playersCount": "opponentTeamPlayersCount"}, inplace=True)

    match_rounds_players_stats_df["fractionOpponentTeamKilled"] = match_rounds_players_stats_df["kills"] / match_rounds_players_stats_df["opponentTeamPlayersCount"]

    match_rounds_players_stats_df["ace"] = (match_rounds_players_stats_df["fractionOpponentTeamKilled"] == 1.0)
    
    match_rounds_players_ace_df = match_rounds_players_stats_df[["roundId", "playerId", "fractionOpponentTeamKilled", "ace"]]

    return match_rounds_players_ace_df

In [30]:
match_rounds_players_ace_df = await get_aces_by_match_id("c5c7500c-6db8-45c8-b6ab-2be01b401ef7")

In [31]:
match_rounds_players_ace_df.head()

,roundId,playerId,fractionOpponentTeamKilled,ace
0,round-1:c5c7500c-6db8-45c8-b6ab-2be01b401ef7,76561198031908632,0.0,False
1,round-1:c5c7500c-6db8-45c8-b6ab-2be01b401ef7,76561198350362302,0.0,False
2,round-1:c5c7500c-6db8-45c8-b6ab-2be01b401ef7,76561198365697700,0.2,False
3,round-1:c5c7500c-6db8-45c8-b6ab-2be01b401ef7,76561198059614402,0.2,False
4,round-1:c5c7500c-6db8-45c8-b6ab-2be01b401ef7,76561198026088156,0.4,False


In [ ]:
async def get_player_kill_player_by_match_id(match_id: str):
    match_rounds_raw_data = await prisma.event.find_many(
        where={
            'eventType': "PLAYER_KILLED_PLAYER",
            'matchId': match_id,
        },
    )